In [1]:
import torch
import numpy as np
from scipy.special import softmax
import os
import glob

In [2]:
def get_pth_files(dataset_path):
   pth_files = []
   for root, dirs, files in os.walk(dataset_path):
       for file in files:
           if file.endswith(".pth"):
               pth_files.append(os.path.join(root, file))
   return pth_files


dataset_path = "/home/rafaelrosendo/Pytorch_trabs/audiocodes2speech_dim0494/train"

pth_files = get_pth_files(dataset_path)
print(pth_files[0])

/home/rafaelrosendo/Pytorch_trabs/audiocodes2speech_dim0494/train/JornalNacional_04_00109.pth


In [3]:
codes_list = []
for pth_file in pth_files:
    #print(pth_file)
    codes = torch.load(pth_file,map_location='cpu')
   # print(codes.shape)
    codes_list.append(codes)
print(codes.shape)

torch.Size([1, 247])


In [4]:
import torch
import torch.nn as nn

code_emb = []
emb_layer = nn.Embedding(8194, 1024)
conv = nn.ConvTranspose1d(in_channels=1024, out_channels=100, kernel_size=2, stride=2, bias=False)

output_array = []

# Colocando em tamanho 100
for codes, pth_file in zip(codes_list, pth_files):
    embeddings = emb_layer(codes.int())

    # Adicionar uma dimensão para corresponder ao formato esperado pela ConvTranspose1d
   # embeddings = embeddings.unsqueeze(0)

    conv_output = conv(embeddings)

    # Redimensionar o tensor de saída para [1, 100, num_aleatorio]
    conv_output = conv_output.squeeze(0).transpose(1, 2)

    # Verificar se os tamanhos dos tensores correspondem
    if output_array:
        if conv_output.size(1) != 2 * embeddings.size(1):
            # Ajustar o tamanho do tensor problemático para corresponder ao tamanho esperado
            conv_output = conv_output[:, :2 * embeddings.size(1), :]
            if conv_output.size(1) != 2 * embeddings.size(1):
                print(f"Descartando tensor problemático: {conv_output.size()}")
                continue

    # Salvar o resultado no array
    output_array.append(conv_output)

# Verificar se há tensores na lista de saída
if len(output_array) == 0:
    print("Nenhum tensor válido encontrado.")
else:
    # Verificar se todos os tensores têm a mesma dimensão 1
    output_size = output_array[0].size(1)
    if all(tensor.size(1) == output_size for tensor in output_array):
        output_tensor = torch.cat(output_array, dim=0)
        print(output_tensor.shape)
    else:
        print("As dimensões dos tensores na lista de saída não correspondem.")

RuntimeError: Given transposed=1, weight of size [1024, 100, 2], expected input[1, 324, 1024] to have 1024 channels, but got 324 channels instead

In [5]:
import torch
import torch.nn as nn

code_emb = []
emb_layer = nn.Embedding(8194, 1024)

output_array = []

# Colocando em tamanho 100
for codes, pth_file in zip(codes_list, pth_files):
    embeddings = emb_layer(codes.int())

    conv = nn.ConvTranspose1d(in_channels=embeddings.size(2), out_channels=100, kernel_size=2, stride=2, bias=False)

    conv_output = conv(embeddings.transpose(1, 2))

    # Redimensionar o tensor de saída para [1, 100, num_aleatorio]
    conv_output = conv_output.transpose(1, 2)

    # Verificar se os tamanhos dos tensores correspondem
    if output_array:
        if conv_output.size(1) != 2 * embeddings.size(1):
            # Ajustar o tamanho do tensor problemático para corresponder ao tamanho esperado
            conv_output = conv_output[:, :2 * embeddings.size(1), :]
            if conv_output.size(1) != 2 * embeddings.size(1):
                print(f"Descartando tensor problemático: {conv_output.size()}")
                continue

    # Salvar o resultado no array
    output_array.append(conv_output)

# Verificar se há tensores na lista de saída
if len(output_array) == 0:
    print("Nenhum tensor válido encontrado.")
else:
    # Verificar se todos os tensores têm a mesma dimensão 1
    output_size = output_array[0].size(1) 
    if all(tensor.size(1) == output_size for tensor in output_array):
        output_tensor = torch.cat(output_array, dim=0)
        print(output_tensor.shape)
    else:
        print("As dimensões dos tensores na lista de saída não correspondem.")

As dimensões dos tensores na lista de saída não correspondem.
